In [1]:
# Import necessary packages
import pandas as pd
import plotly.express as px

In [2]:
# Read in Data
batting = pd.read_csv('Batting.csv')
master = pd.read_csv('Master.csv', encoding='latin-1')

In [3]:
# Replace NAs with 0s for certain columns
batting['SF'] = batting['SF'].fillna(0)
batting['HBP'] = batting['HBP'].fillna(0)

In [4]:
#Creaete functions to get playerid, player birth year, and stats for any player
def getid(firstname, lastname):
    df = master.loc[(master['nameFirst'] == firstname) & (master['nameLast'] == lastname)]
    id = df['playerID']
    return id

def getbirthyear(playerid):
    df = master.loc[master['playerID'] == playerid]
    if df['birthMonth'].any() >= 7:
        df['birthMonth']
    else:
        df['birthMonth'] += 1
    return df['birthYear']

def getstats(playerid):
    df = batting.loc[batting['playerID'] == playerid]
    birthyear = getbirthyear(playerid)
    df['Age'] = df['yearID'] - int(birthyear)
    df['SLG'] = (df['H'] - df['2B'] - df['3B'] - df['HR'] + 2 * df['2B'] + 3 * df['3B'] + 4 * df['HR']) / df['AB']
    df['OBP'] = (df['H'] + df['BB']) / (df['H'] + df['AB'] + df['BB'] + df['SF'])
    df['OPS'] = df['SLG'] + df['OBP']
    stats = df[['Age', 'SLG', 'OBP', 'OPS']]
    return stats

In [ ]:
mantle = getstats('mantlmi01')
mantle

In [7]:
# Create plot showing OPS per year over Mickey Mantle's career
mantleplot = px.scatter(mantle, x = 'Age', y = 'OPS')
mantleplot.show()

In [8]:
# Fitting Quadratic Regression
import numpy as np
import plotly.graph_objs as go
model = np.poly1d(np.polyfit(mantle['Age'], mantle['OPS'], 2))

In [21]:
# Import necessary modules
import plotly.offline as po
import numpy as np
import plotly.graph_objs as go
# Create Quadratic Regression Model
model = np.poly1d(np.polyfit(mantle['Age'], mantle['OPS'], 2))
x = np.linspace(1, 50, 50)
y = model(x)
# Create graphs showing quadratic model fitted onto graph of Age vs. OPS to see where Mantle peaked according to model
mantleplot = go.Scatter(
    x = mantle['Age'],
    y = mantle['OPS'],
    mode = 'markers',
    marker = go.Marker(color='rgb(255, 127, 14)'),
    name = 'Data'
)
mantleline = go.Scatter(
    x = x,
    y = y,
    mode = 'lines',
    marker = go.Marker(color = 'rgb(31, 119, 180)'),
    name = 'Fit'
)
data = [mantleplot, mantleline]
fig = go.Figure(data = data)
fig.update_yaxes(range = [0, 1.5])
fig.update_xaxes(range = [10, 45])
# Highlight Maximum Values for Age and OPS to understand Mickey Mantle's Peak
max_y = int(max(y))
max_x = (x[y.argmax(max_y)])
fig.add_vline(x = max_x, line_dash = 'dash', line_color = 'grey')
fig.add_hline(y = max(y), line_dash = 'dash', line_color = 'grey')
fig.add_annotation(x = 29.8, y = 0.05, text = 'Peak Age = 28', showarrow = False)
po.plot(fig)

'temp-plot.html'

In [638]:
# Create function to make fitted model / plot  to understand player peaks for any player
def careerprojchart(playerid):
    stats = getstats(playerid)
    model = np.poly1d(np.polyfit(stats['Age'], stats['OPS'], 2))
    x = np.linspace(1, 50, 50)
    y = model(x)
    plot = go.Scatter(
        x = stats['Age'],
        y = stats['OPS'],
        mode = 'markers',
        marker = go.Marker(color='rgb(255, 127, 14)'),
        name = 'Data'
    )
    line = go.Scatter(
        x = x,
        y = y,
        mode = 'lines',
        marker = go.Marker(color = 'rgb(31, 119, 180)'),
        name = 'Fit'
    )
    data = [plot, line]
    fig = go.Figure(data = data)
    fig.update_yaxes(range = [0.5, 1.1])
    fig.update_xaxes(range = [18, 45])
    return fig

In [ ]:
careerprojchart('bondsba01')

In [113]:
# Read in fielding data for Similarity Scores
fielding = pd.read_csv('Fielding.csv')

In [ ]:
# Create dataframe that shows us each player's most played position for position element of Similarity Score
sumfield = fielding.groupby(['playerID', 'POS'], as_index = False).sum()
playerpos = sumfield.groupby(['playerID', 'G', 'POS'], as_index = False).size().reset_index().drop(['index', 'size'], axis = 1)
playerpos = playerpos.loc[playerpos.reset_index().groupby(['playerID'])['G'].idxmax()]
playerpos = playerpos.drop(['G'], axis = 1).reset_index()
playerpos

In [612]:
# Create dataframe of necessary information for Similarity Scores
totals = batting.groupby(['playerID'], as_index = False)[['G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'SB']].sum()
totals['1B'] = totals['H'] - (totals['2B'] + totals['3B'] + totals['HR'])

In [613]:
# Create AVG and SLG columns for Similarity scores and drop 1B stat (it is not counted in the Similarity Score)
totals['AVG'] = totals['H'] / totals['AB']
totals['SLG'] = (totals['1B'] + (totals['2B'] * 2) + (totals['3B'] * 3) + (totals['HR'] * 4)) / totals['AB']
totals = totals.drop(['1B'], axis = 1)


In [614]:
# Merge playerpos and totals dataframes for Similarity Scores
totals = pd.merge(playerpos, totals, on = 'playerID')

In [615]:
# Create Dictionary of positional values for Sim Scores and add these values to totals dataframe
PosValue = {
    'C': 240,
    'SS': 168,
    '2B': 132,
    '3B': 84,
    'OF': 48,
    'LF': 48,
    'CF': 48,
    'RF': 48,
    '1B': 12,
    'P': 0,
    'DH': 0
}
totals['PosValue'] = totals['POS']
totals['PosValue'] = totals['PosValue'].replace(PosValue)

In [628]:
# Create function to get Similarity Scores and then compare these scores to input player and top 10 most similar players
def similarityscore(playerid):
    global totals
    player_to_compare = totals.loc[totals['playerID'] == playerid]
    totals['SScore'] = (
        1000 - 
        np.floor(abs(totals.iloc[0:, 3].sub(player_to_compare.iloc[0, 3])) / 20) -
        np.floor(abs(totals.iloc[0:, 4].sub(player_to_compare.iloc[0, 4])) / 75) -
        np.floor(abs(totals.iloc[0:, 5].sub(player_to_compare.iloc[0, 5])) / 10) -
        np.floor(abs(totals.iloc[0:, 6].sub(player_to_compare.iloc[0, 6])) / 15) - 
        np.floor(abs(totals.iloc[0:, 7].sub(player_to_compare.iloc[0, 7])) / 5) - 
        np.floor(abs(totals.iloc[0:, 8].sub(player_to_compare.iloc[0, 8])) / 4) -
        np.floor(abs(totals.iloc[0:, 9].sub(player_to_compare.iloc[0, 9])) / 2) -
        np.floor(abs(totals.iloc[0:, 10].sub(player_to_compare.iloc[0, 10])) / 10) -
        np.floor(abs(totals.iloc[0:, 11].sub(player_to_compare.iloc[0, 11])) / 25) -
        np.floor(abs(totals.iloc[0:, 12].sub(player_to_compare.iloc[0, 12])) / 150) -
        np.floor(abs(totals.iloc[0:, 13].sub(player_to_compare.iloc[0, 13])) / 20) -
        np.floor(abs(totals.iloc[0:, 14].sub(player_to_compare.iloc[0, 14])) / 0.001) -
        np.floor(abs(totals.iloc[0:, 15].sub(player_to_compare.iloc[0, 15])) / 0.002) -
        abs(totals.iloc[0:, 16].sub(player_to_compare.iloc[0, 16]))    
    )    
    totals = totals.sort_values(by = ['SScore'], ascending = False)
    return totals[0:11][['playerID', 'SScore']]

In [629]:
similarityscore('mantlmi01')

,playerID,SScore
9860,mantlmi01,1000.0
10068,matheed01,853.0
14262,schmimi01,848.0
14568,sheffga01,847.0
15901,thomafr04,844.0
15076,sosasa01,831.0
13063,ramirma02,829.0
541,bagweje01,818.0
15925,thomeji01,817.0
12077,ottme01,817.0


In [640]:
careerprojchart('sosasa01')

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_6952\1948160369.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_6952\1948160369.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_6952\1948160369.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [ ]:
getid('Derek', 'Jeter')


In [630]:
similarityscore('jeterde01')

,playerID,SScore
7844,jeterde01,1000.0
17630,yountro01,863.0
1211,biggicr01,829.0
222,alomaro01,796.0
5632,gehrich01,762.0
5257,francju01,758.0
11019,molitpa01,755.0
3644,damonjo01,737.0
5350,friscfr01,733.0
13630,rodriiv01,729.0
